<a href="https://colab.research.google.com/github/AgustinBatistelli/programacion_concurrente/blob/master/Ejercicios_2_MPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programacion Concurrente TP2 Parte2 MPI

Para este ejercicio se ha optado por aplicar el tema teórico **MPI** (Message Passing Interface). La finalidad del ejercicio es ampliar el conocimiento sobras la manera que posee Python para implementar la comunicación entre distintos procesos con el uso de una librería denominada **MPI4py** [1]. Para más información puede consultar la última revisión en [2].

Este ejercicio fue presentado por alumnos del curso 2020.

---
## 2.1. Ejercicio - Hola Mundo con MPI


### 2.1.1. Preguntas del ejercicio

Ejecute este ejemplo para $4$ o más instancias y responda:

a) ¿Qué función realiza la instancia maestra? ¿Qué función realizan las instancias esclavas?

b) ¿Cuántas de esas instancias ejecuta la función `main()`, `initWork()` y `doWork()`?

c) ¿Cómo se diferencian los mensajes de trabajo o de finalización?

d) ¿Cómo implementaría la función BLAS `axpy()` con este patrón?¿Sería eficiente? *Tips*: Pide solo el planteo, no la implementación.

e) ¿Qué sucede cuando solo ejecuta con una sola instancia?

f) *Punto opcional*: El código que ejecutan las instancias esclavas, tienen un error en su lógica. ¿Cómo se podría solucionar?

### 2.1.1. Respuestas del ejercicio

a) La instancia maestra (jefe) se encarga de enviar tareas por realizar junto con un tiempo de descanso a las instancias esclavas (empleados). Si resulta que hay más tareas por realizar, el jefe irá asignando a los empleados las siguientes tareas a realizar a los empleados a medida que estos vayan informándole sobre la finalización de las tareas.

b) La función main se ejecuta en cada una de las instancias, cambiando su comportamiento dependiendo de si me encuentro en la instancia maestra o en una de las instancias esclavas. Luego de eso, el método initWork será ejecutado solo por una de las instancias pasadas por parámetro (la instancia 0, que será el jefe). El resto de instancias desde la 1 hasta la N-1 (siendo N el parámetro enviado) ejecutarán el método doWork. Resumiendo: main ejecuta N veces, initWork ejecuta 1 vez, doWork ejecuta N-1 veces.

c) Utilizando las constantes WORK_FLAG (valor 1) y END_WORK_FLAG (valor 2). La instancia jefe les envía a los empleados el WORK_FLAG al comenzar o cuando los empleados vayan terminando una tarea y sea necesario asignarles otra. Una vez que los empleados ya finalizaron sus tareas, el jefe les envía el END_WORK_FLAG, lo cual les permite a los empleados saber que tienen que finalizar y marcar la tarjeta. Es con este flag que los empleados saben que tienen que finalizar o no su ejecución.

d) La función axpy consistía en realizar la multiplicación del vector **x** por el escalar **a**, y luego de eso, sumar ese vector resultado al vector **y**. Podría utilizar las funciones de scatter para distribuir los datos en los procesos, creando una cantidad N de procesos (donde N sea la cantidad de elementos del vector) y enviando a cada proceso el valor de uno de los elementos de x, hacer que los multipliquen por el valor de a (que se podría enviar con un broadcast) y con un gather reunirlos de vuelta en el proceso principal donde serían sumados a los elementos de y.

En pseudocódigo podría ser algo así:
```
BroadCast(a)
Scatter(x)
x*a
Gather(x*a)
y = x + y
```

No sería demasiado eficiente crear una cantidad X de procesos (MPI no pueda usar hilos) para que cada uno de estos ejecute un único dato y luego lo envíe al principal. Siendo que Open MP trabaja con hilos, se ve más eficiente utilizarlo, y tras ver los resultados del tp anterior, realizar esta función de forma secuencial es incluso todavía más óptimo.

e) La primera instancia en ejecución siempre será la que representa a la instancia maestra (el jefe con id=0). Si se le indica por parámetro que el programa ejecutará con una sola instancia, entonces esa instancia será la instancia maestra. Al solamente ejecutar esa instancia, no puede crear las tareas dado a que no tiene un número de empleados a los cuales asignarles dicha instancias. Lo único que hace es llamar a la función initWork, que depende del número de procesos empleados, que será de 0.

f) Cuando el jefe manda a los empleados el END_WORK_FLAG, estos muestran por pantalla que recibieron un nuevo trabajo y que les corresponden un total de 0 horas de descanso, pero luego de eso, realizan una pregunta para saber si el tag recibido por el jefe es igual al Flag de finalización de trabajo, y luego de eso indican su finalización en caso de que sea verdad. Debería realizar esa pregunta antes de que muestre por pantalla que tiene una nueva tarea, de esa manera no enviaría el print de que recibió una nueva tarea con 0 horas de descanso cuando se le está indicando que debe finalizar.

###2.1.2. Armado del ambiente

Es en donde se instalar, por única vez, el módulo MPI4py de Python en el cuaderno.

In [ ]:
! pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=3365670 sha256=1bbc905fb9c592c71f5e0fd307b285b073aca5e1397a2276b3b78ddd1dc2dd6c
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py


### 2.1.3. Código del programa en Lenguaje Python

In [ ]:
%%writefile Ejercicio2.py
from mpi4py import MPI
import numpy as np
import time

# --------------------------------------------
# Formulario
Max_tiempo_sleep =   5#@param {type: "slider", min: 1, max: 10}
Min_tiempo_sleep =   0#@param {type: "slider", min: 0, max: 10}
# --------------------------------------------

# --------------------------------------------
# Constantes de comunicacion
WORK_FLAG = 1
END_WORK_FLAG = 2
# --------------------------------------------

def main():
    comm = MPI.COMM_WORLD # Instanciamos el tipo de comunicador a utilizar.
    id = comm.Get_rank()  # Obtenemos el id como atributo del proceso que se ejecuta.

    # Utilizamos el 0 para definir al procesos Maestro, cualquier otro id sera un esclavo.
    if (id == 0) :
        init() # Llamamos funcion init para eventos que requeriremos inicialmente solo 1 vez.
        numProcesses = comm.Get_size()  # Obtenemos el numero de procesos totales ejecutados.
        numTasks = (numProcesses-1)*4 # Se setea el numero de tareas.
        workTimes = generateTasks(numTasks) # Se generan las tareas, en este caso seran
        print("El jefe crea {} horas de descanso de sus empleados:".format(workTimes.size), flush=True)
        print(workTimes, flush=True)
        initWork(comm, workTimes, numProcesses)
    else:
        doWork(comm)

def generateTasks(numTasks):
    #TODO: Cambiar la semilla del random para que se generen efectivamente diferentes numeros.
    np.random.seed(1000)
    return np.random.randint(low=Min_tiempo_sleep, high=Max_tiempo_sleep, size=numTasks)

def init():
  print()
  print("Version MPI4py utilizada: {}".format(MPI.Get_version()), flush=True)
  print()
  print( "------------------------------------", flush=True)
  print( "Sistema de trabajo Suizo:", flush=True)
  print( "------------------------------------", flush=True)
  print()

def initWork(comm, workTimes, numProcesses):
    totalWork = workTimes.size
    workcount = 0
    recvcount = 0

    print("Jefe enviando las tareas iniciales:", flush=True)
    for id in range(1, numProcesses):
        if workcount < totalWork:
            work=workTimes[workcount]
            comm.send(work, dest=id, tag=WORK_FLAG) # Envia mensaje de iniciar trabajo con el dato correspondiente del array.
            workcount += 1
            print("Jefe envia trabajo y {} hs de descanso al empleado {}.".format(work, id), flush=True)
    print( "------------------------------------", flush=True)

    # Mientras haya trabajo, se recibe el resultado de los empleados y se sigue enviando MAS trabajo.
    while (workcount < totalWork) :
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat) # Recivimos resultados de los empleados.
        recvcount += 1
        workerId = stat.Get_source() # Obtenemos el identificador del empleado.
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)
        #send next work
        work=workTimes[workcount]
        comm.send(work, dest=workerId, tag=WORK_FLAG)
        workcount += 1
        print("Jefe envia nuevo trabajo y {} hs de descanso al empleado {}.".format(work, workerId), flush=True)

    while (recvcount < totalWork):
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat)
        recvcount += 1
        workerId = stat.Get_source()
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)

    for id in range(1, numProcesses):
        comm.send(0, dest=id, tag=END_WORK_FLAG)


def doWork(comm):
    while(True):
        stat = MPI.Status() # Obtiene el estado actual del empleado.
        waitTime = comm.recv(source=0, tag=MPI.ANY_TAG, status=stat) # Obtiene lo enviado por el Jefe.
        print("Soy el empleado con id {}, toca descanzo por {} hs.".format(comm.Get_rank(), waitTime), flush=True)

        if (stat.Get_tag() == END_WORK_FLAG):
            print("Marca tarjeta el empleado {}.".format(comm.Get_rank()), flush=True)
            return
        time.sleep(waitTime)
        comm.send(waitTime, dest=0)

main()


Overwriting Ejercicio2.py


### 2.1.4 Ejecución del programa

In [ ]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }
NRO =   1#@param {type: "number"}
# --------------------------------------------

! mpirun --allow-run-as-root --oversubscribe -np $NRO python Ejercicio2.py


Version MPI4py utilizada: (3, 1)

------------------------------------
Sistema de trabajo Suizo:
------------------------------------

El jefe crea 0 horas de descanso de sus empleados:
[]
Jefe enviando las tareas iniciales:
------------------------------------


---
# Bibliografía

[1] MPI4py: https://mpi4py.readthedocs.io/en/stable/

[2] La versión oficial de MPI (Versión 4): https://www.mpi-forum.org/docs/mpi-4.0/mpi40-report.pdf

